### Import Libraries

In [8]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

### Loading and Preprocessing data

In [9]:
# Define the paths to the dataset folders
data_path = r'C:\Users\ugoyal\Documents\Image-Detection\data'
non_vehicle_folder = os.path.join(data_path, 'non-vehicles')
vehicle_folder = os.path.join(data_path, 'vehicles')

# Initialize lists to hold image data and labels
images = []
labels = []

# Counters for labeling
car_count = 0
non_car_count = 0

# Helper function to process images from a given folder
def process_images_from_folder(folder_path, is_vehicle):
    global car_count, non_car_count
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if the image couldn't be read
        
        # Resize image to 224x224 pixels
        img = cv2.resize(img, (224, 224))
        images.append(img)
        
        # Assign labels based on whether it's a vehicle or non-vehicle
        if is_vehicle:
            label = 1  # Label for car
            car_count += 1
        else:
            label = 0  # Label for non-car
            non_car_count += 1
        
        labels.append(label)

# Process images from both folders
process_images_from_folder(vehicle_folder, is_vehicle=True)
process_images_from_folder(non_vehicle_folder, is_vehicle=False)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels:", labels)


Images shape: (17760, 224, 224, 3)
Labels: [1 1 1 ... 0 0 0]


### Normalizing and Splitting data

In [10]:
# Normalize the images
images = images / 255.0

# Convert labels to categorical
labels = to_categorical(labels, num_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

### Building the Model

CNN Model experimenting with the potential benefits by simply adding layers

In [11]:
# Initialize the model
model = Sequential()

# Add an Input layer as the first layer in the model
model.add(Input(shape=(224, 224, 3)))

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,397,122 (169.36 MB)

 Trainable params: 44,397,122 (169.36 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Initialize the image data generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Fit the data generator on the training data
datagen.fit(X_train)

### Training the Model

In [13]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model using the data generator
history = model.fit(datagen.flow(X_train, y_train, batch_size=128), 
                    epochs=20, 
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

Epoch 1/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 208s 456ms/step - accuracy: 0.8026 - loss: 0.4895 - val_accuracy: 0.9051 - val_loss: 0.2125
Epoch 2/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 189s 423ms/step - accuracy: 0.9539 - loss: 0.1348 - val_accuracy: 0.9775 - val_loss: 0.0593
Epoch 3/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 188s 420ms/step - accuracy: 0.9634 - loss: 0.1011 - val_accuracy: 0.9699 - val_loss: 0.0890
Epoch 4/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 188s 421ms/step - accuracy: 0.9633 - loss: 0.1017 - val_accuracy: 0.9840 - val_loss: 0.0460
Epoch 5/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 186s 416ms/step - accuracy: 0.9699 - loss: 0.0865 - val_accuracy: 0.9837 - val_loss: 0.0548
Epoch 6/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 186s 416ms/step - accuracy: 0.9682 - loss: 0.0945 - val_accuracy: 0.9870 - val_loss: 0.0416
Epoch 7/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 188s 420ms/step - accuracy: 0.9740 - loss: 0.0769 - val_accuracy: 0.9868 - val_loss: 0.0402
Epoch 8/50
444/444 ━━━━━━━━━━━━━━━━━━━━ 186s 416ms/step - accuracy: 0.9741 -

KeyboardInterrupt: 